# Active work in LIFE Target Database

In [5]:
import numpy as np
b=0.000004
print(-1.74532598e-04)

-0.000174532598


## final plot simplification

In [ ]:
import astropy.io.votable
from utils.io import load
#load starcat4
[starcat]=load(['StarCat4'])


In [ ]:
print(starcat.colnames)

In [ ]:
import importlib
import numpy as np
import analysis as lta
importlib.reload(lta)#reload module after changing it
#plot starcat4
#lta.final_plot([starcat['sptype_string','dist_st_value']],['StarCat4'])
stars=[starcat['sptype_string','dist_st_value'],
       starcat['sptype_string','dist_st_value'][np.where(starcat['binary_flag']=='True')],
      starcat['sptype_string','dist_st_value'][np.where(starcat['binary_flag']=='False')]]
spectral_type_samples=[starcat['sptype_string'],
                      starcat['sptype_string'][np.where(starcat['binary_flag']=='True')],
                      starcat['sptype_string'][np.where(starcat['binary_flag']=='False')]]
distance_sample=[starcat['dist_st_value'],
                starcat['dist_st_value'][np.where(starcat['binary_flag']=='True')],
                starcat['dist_st_value'][np.where(starcat['binary_flag']=='False')]]
lables=['StarCat4 Total','Binaries','Single Stars']
lta.final_plot(stars,spectral_type_samples,distance_sample,lables)
#I am currently in the middle of replacing stars with spectral_type_samples and distance_samples

In [ ]:
importlib.reload(lta)#reload module after changing it

lta.spectral_type_histogram_catalog_comparison(spectral_type_samples,distance_sample,lables)

In [ ]:
import importlib
import analysis as lta
importlib.reload(lta)#reload module after changing it
#plot starcat4
#lta.final_plot([starcat['sptype_string','dist_st_value']],['StarCat4'])
stars=[starcat['sptype_string','dist_st_value'],
       starcat['sptype_string','dist_st_value'][np.where(starcat['binary_flag']=='True')],
      starcat['sptype_string','dist_st_value'][np.where(starcat['binary_flag']=='False')]]
spectral_type_samples=[starcat['sptype_string'],
                      starcat['sptype_string'][np.where(starcat['binary_flag']=='True')],
                      starcat['sptype_string'][np.where(starcat['binary_flag']=='False')]]
distance_sample=[starcat['dist_st_value'],
                starcat['dist_st_value'][np.where(starcat['binary_flag']=='True')],
                starcat['dist_st_value'][np.where(starcat['binary_flag']=='False')]]
lables=['StarCat4 Total','Binaries','Single Stars']
lta.final_plot(stars,spectral_type_samples,distance_sample,lables)
#I am currently in the middle of replacing stars with spectral_type_samples and distance_samples

## for loops

In [ ]:
numbers = [1, 2, 3, 4, 5]
for i in range(len(numbers)):
    print(numbers[i])

In [ ]:
for i, number in enumerate(numbers):
    print(i, number)
    #benefit: you don't need range(len()), also don't need to do number[i]

In [ ]:
for number in numbers:
    print(number)
#benefit, if you don't really need the index and you can get rid of range, and len

In [ ]:
import numpy as np
import astropy as ap

#might be useful for replace value function. I don't need index there.
#though I actually don't need anything there. so won't make stuff easier
def replace_value(cat,column,value,replace_by):
    
    cat[column][np.where(cat[column]==value)]= \
            [replace_by for i in range(
        len(cat[column][np.where(cat[column]==value)]))]
    return cat

def replace_value_enumerate(cat,column,value,replace_by):
    
    cat[column][np.where(cat[column]==value)]= \
            [replace_by for entries in enumerate(cat[column][np.where(cat[column]==value)])]
    return cat

#test data set
sptype=np.array(['M3.51','M3:','dM5.0','M5.0IV','','D2.3V','G2VI'])
main_id=np.array(['G 227-48B','* mu. Dra C','FBS 1415+456','test','test2','test3','test4'])
temp=ap.table.Table((main_id,sptype),names=('main_id','sptype_string' ))

print(temp)
new=replace_value(temp,'sptype_string','dM5.0','M5.0V')
print(new)
en=replace_value_enumerate(temp,'sptype_string','dM5.0','M5.0V')
print(en)

#fazit, works but not worth the trouble here

In [ ]:
#maybe useful in different location e.g.
def best_para(para,mes_table):
    """
    This function keeps only highest quality row for each object. 
    
    tried to avoid . for performance reasons
    
    :param para: Describes parameter e.g. mass
    :type para: str
    :param mes_table: Contains only columns 'main_id',
         para+'_value',para+'_err',para+'_qual' and para+'_ref'
    :type mes_table: astropy.table.table.Table
    :returns: Table like mes_table but only highest quality rows for
         each object
    :rtype: astropy.table.table.Table
    """
    
    if para=='id':
        best_para=mes_table[:0].copy()
        grouped_mes_table=mes_table.group_by('id_ref')
        mask = grouped_mes_table.groups.keys['id_ref'] == '2000A&AS..143....9W'
        best_para=grouped_mes_table.groups[mask]
        # TBD: use id_ref as variable from provider_bibcode 
        #        instad of constant""")
        for ref in ['2022A&A...664A..21Q','2016A&A...595A...1G','priv. comm.',
                    '2020A&C....3100370A','2001AJ....122.3466M']:
            mask = grouped_mes_table.groups.keys['id_ref'] == ref
            new_ids=grouped_mes_table.groups[mask][np.where(np.invert(np.in1d(
                                        grouped_mes_table.groups[mask]['id'],
                                        best_para['id'])))]
        best_para=vstack([best_para,new_ids])
        best_para.remove_column('id_ref')
        return best_para
    elif para=='membership':
        best_para=mes_table[:0].copy()
        grouped_mes_table=mes_table.group_by(['child_object_idref',
                                              'parent_object_idref'])
        ind=grouped_mes_table.groups.indices
        for i in range(len(ind)-1):
            l=ind[i+1]-ind[i]
            if l==1:
                best_para.add_row(grouped_mes_table[ind[i]])
            else:
                temp=grouped_mes_table[ind[i]:ind[i+1]]
                not_nan_temp=temp[np.where(temp[para]!=999999)]
                if len(not_nan_temp)>0:
                    max_row=not_nan_temp[np.where(
                            not_nan_temp[para]==max(not_nan_temp[para]))]
                    for j in range(ind[i],ind[i+1]):
                        if grouped_mes_table[para][j]==max(not_nan_temp[para]):
                            best_para.add_row(grouped_mes_table[j])
                            break # make sure not multiple of same max 
                                   # value are added
                else:
                    #if none of the objects has a membership entry 
                    # then pick just first one
                    best_para.add_row(grouped_mes_table[ind[i]])
        return best_para
    elif para=='binary':
        columns=['main_id',para+'_flag',para+'_qual',para+'_source_idref']
    elif para=='mass_pl':
        columns=['main_id',para+'_value',para+'_rel',para+'_err',para+'_qual',
                para+'_source_idref']
    elif para=='sep_ang':
        columns=['main_id',para+'_value',para+'_err',para+'_obs_date',
                para+'_qual',para+'_source_idref']
    else:
        columns=['main_id',para+'_value',para+'_err',para+'_qual',
                para+'_source_idref']
    mes_table=mes_table[columns[0:]]
    best_para=mes_table[columns[0:]][:0].copy()
    #group mes_table by object (=main_id)
    grouped_mes_table=mes_table.group_by('main_id')
    #take highest quality
    #quite time intensive (few minutes) could maybe be optimized using 
    # np.in1d function
    for j in range(len(grouped_mes_table.groups.keys)):
    # go through all objects
        for qual in ['A','B','C','D','E','?']:
            for i in range(len(grouped_mes_table.groups[j])):
                if grouped_mes_table.groups[j][i][para+'_qual']==qual:
                    best_para.add_row(grouped_mes_table.groups[j][i])
                    break # if best para found go to next main_id group
            else:
                continue # only executed if the inner loop did NOT break
            break  # only executed if the inner loop DID break 
    return best_para

## automatic download of disk data 

In [ ]:
import astropy as ap #votables
import requests
from datetime import datetime

gk_provider=ap.table.Table()
gk_provider['provider_url']=['http://drgmk.com/sdb/samples/disks_30pc_/']
gk_provider['provider_access']=datetime.now().strftime('%Y-%m-%d')
filename=additional_data_path+'sdb_30pc_'+gk_provider['provider_access'][0]+'.xml'
URL = gk_provider['provider_url'][0]

#pulling xml content down
response = requests.get(URL)
#create a file
with open(filename, 'wb') as file:
    #saving the content into the file
    file.write(response.content)

In [ ]:
#so far not working because of authentification issue

In [ ]:
print(ap.io.votable.parse_single_table(filename).to_table())

In [ ]:
import numpy as np #arrays
import pyvo as vo #catalog query
import astropy as ap #votables
from datetime import datetime
import importlib #reloading external functions after modification

#self created modules
import helperfunctions as hf
importlib.reload(hf)#reload module after changing it
import provider as p
importlib.reload(p)#reload module after changing it

table_names=['sources','objects','provider','ident','h_link','star_basic',
              'planet_basic','disk_basic','mes_mass_pl',
              'mes_teff_st','mes_radius_st','mes_mass_st','mes_binary','mes_sep_ang']
empty_provider=[ap.table.Table() for i in range(len(table_names))]

gk=p.provider_gk(table_names,empty_provider[:],30)
#maybe I have some masked entries in the wds_ident due to system_main_id. but column types are not masked columns
#having the code snipped below not included makes the code run.
#for col in ['system_main_id','primary_main_id','secondary_main_id']:
         #   wds[col][np.where(wds[col]=='')]=np.ma.masked
#this code is needed as when I save the wds stuff the masked entries get transformed into '' so when I load the wds
#data I need to transform it back.
#quite a few thousends of rows are affected by this
#found out I have different mask type assigned np.ma.core.MaskedConstant vs np.ma.masked
#when changign that I get different error message. maby issue is that one of them is str and other object?
#ah one is the type and the other is the entry
#I think I found another error, I should not have system_main_id being masked -> but I have 4000 that are!
#also having considerable issues with my mouse currently
#trying to find out where in wds creation this happens
#apparently in primary and secondary cut
#now did assign system_name where system_main_id masked but there are still some left, why?

#I overhauled a lot, testing if it works
#mes_sep uniqueness issue
#changed stuff because system_main_id could be masked
#still having issue how to get uniqueness without conflict of masked 
#observation dates entries -> two uniqueness runs


#also included ids_from_ident in exo and gaia, test if that works

## another_multifix

### spectral type with + but no binary

In [ ]:
import numpy as np

cat_sb=database_tables[table_names.index('star_basic')]
print(cat_sb['sptype_string','class_temp','class_lum'][np.where(cat_sb['main_id']=='V* HN Peg')])

## hwo cat4 comparison

In [ ]:
#check why WDS J01441-1556 (main id * tau Cet) has no sep in db despite finding it in wds provider via topcat
#topcat -> sy WDS J01441-1556 as parent of WDS J01441-1556A and WDS J01441-1556B, with seps
import numpy as np #arrays
import pyvo as vo #catalog query
import astropy as ap #votables
from datetime import datetime
import importlib #reloading external functions after modification

#self created modules
import helperfunctions as hf
importlib.reload(hf)#reload module after changing it
import provider as p
importlib.reload(p)#reload module after changing it

table_names=['sources','objects','provider','ident','h_link','star_basic',
              'planet_basic','disk_basic','mes_mass_pl',
              'mes_teff_st','mes_radius_st','mes_mass_st','mes_binary','mes_sep_ang']
empty_provider=[ap.table.Table() for i in range(len(table_names))]


#life db currently:
provider=database_tables
#provider=wds

cat_i=provider[table_names.index('ident')]
cat_o=provider[table_names.index('objects')]
cat_h=provider[table_names.index('h_link')]
cat_sb=provider[table_names.index('star_basic')]
#cat_sb=provider[table_names.index('mes_sep_ang')]

#need to rewrite this below to find out concrete reason for why objects in list don't have
# passed both components in list criteria
object_not_found=[]

for name in l:
    if len(cat_i['main_id'][np.where(cat_i['id']==name)])>0:
        main_id=cat_i['main_id'][np.where(cat_i['id']==name)][0]
        #print(cat_o['main_id','type'][np.where(cat_o['main_id']==main_id)])
        if len(cat_h[np.where(cat_h['parent_main_id']==main_id)])>0:
                print('object with found child object')
                print(cat_h[np.where(cat_h['parent_main_id']==main_id)])
                if len(cat_h[np.where(cat_h['parent_main_id']==main_id)])>0:
                    #print(cat_sb['main_id','binary_flag'][np.where(cat_sb['main_id']==main_id)])
                    print(cat_sb['main_id','sep_ang_value'][np.where(cat_sb['main_id']==main_id)])
                    for child in list(cat_h['child_main_id'][np.where(cat_h['parent_main_id']==main_id)]):
                        print(cat_sb['main_id','sep_ang_value','binary_flag'][np.where(cat_sb['main_id']==child)])
        else:
            
    else:
        object_not_found.append(name)

In [ ]:
#this code sorts out the reason why objects were not included in the cat 4

#not in query
l_cat3=['HD  15468' ,'LP  704-15' ,'NAME BD-21 1074BC' ,'*  39 Tau' ,'* alf Com',
 '* alf Gem' ,'* eps Eri' ,'* iot Hor' ,'* m Tau' ,'2MASS J04291842-3123568',
 'AG+10  687' ,'BD+01  2684' ,'BD+10  1032' ,'BD+12  1944' ,'BD+13  2618',
 'BD+19  3268' ,'BD+39  2376' ,'BD+45  4378' ,'BD+46  1889' ,'BD+67   552',
 'BD-09   956' ,'BD-15  6290' ,'BPS CS 31060-0015' ,'CD-26  3506',
 'CD-28   302' ,'CD-35  2213' ,'CD-36  6589' ,'CD-38 13254' ,'CD-40  5404',
 'CD-44  4756' ,'HD 179930' ,'CD-58   538' ,'CD-68    47' ,'CD-72  1700',
 'FBS 0444-113' ,'G  19-15' ,'G  24-16' ,'G  41-14' ,'G  41-8' ,'G  62-28',
 'G  68-5' ,'G  69-47' ,'G  78-28' ,'G  96-21' ,'G  96-45' ,'G 100-28',
 'G 100-50' ,'G 119-36' ,'G 131-26' ,'G 139-12' ,'G 139-3' ,'G 140-9' ,'G 152-1',
 'G 155-29' ,'G 158-50' ,'G 161-7' ,'G 161-74' ,'G 165-61' ,'G 172-1',
 'G 179-20' ,'G 179-33' ,'G 179-57' ,'G 193-27' ,'G 194-47' ,'G 195-55',
 'G 207-16' ,'G 208-44' ,'G 217-32' ,'G 221-24' ,'G 224-57' ,'G 237-64',
 'G 237-78' ,'G 248-32' ,'G 250-29' ,'HD  15285' ,'HD  26347' ,'HD  32450',
 'HD  50281B' ,'HD  57095' ,'HD  76740' ,'HD  79555' ,'HD  79969' ,'HD 115953',
 'HD 120476' ,'HD 135204' ,'HD 144253' ,'HD 150848' ,'HD 155876' ,'HD 156384',
 'HD 172051' ,'HD 182488' ,'HD 196795' ,'HD 197092' ,'HD 206804' ,'HD 222834A',
 'HD 281621' ,'L   43-72' ,'L  119-21' ,'L  186-67' ,'L  345-91' ,'L  387-102',
 'L  433-153' ,'L  440-30' ,'L  465-84' ,'L  601-78' ,'L  671-20' ,'L  737-9',
 'L  820-19' ,'LP   44-162' ,'LP   44-334' ,'LP  206-11' ,'LP  275-68',
 'LP  456-27' ,'LP  467-16' ,'LP  497-33' ,'LP  518-58' ,'LP  715-51',
 'LP  869-26' ,'LP  889-37' ,'LSPM J0339+2457' ,'MCC 99' ,'MCC 253' ,'MCC 568',
 'PM J02133+3648' ,'PM J07349+1445' ,'Ross   28' ,'Ross   29' ,'Ross   52',
 'Ross   59' ,'Ross  188' ,'Ross  265' ,'BD-12  2918' ,'Ross  708' ,'Ross  987',
 'Ross 1063' ,'SCR J0533-4257' ,'SCR J0630-7643' ,'SCR J1546-5534' ,'StM 187',
 'TWA 22' ,'APMPM J0710-5704' ,'PM J00066-0705' ,'1RXS J063531.2-040314',
 'PM J20300+0023' ,'LSPM J0553+2507' ,'V* CV Cnc' ,'V* DG CVn' ,'V* EZ Aqr',
 'V* SZ UMa' ,'V* V2367 Oph' ,'V* V484 Tau' ,'UPM J1055-7356' ,'WT 1827',
 'WT 460' ,'Wolf   66' ,'Wolf  230' ,'Wolf  358' ,'Wolf  414' ,'Wolf  424',
 'Wolf  922' ,'Wolf 1062' ,'Wolf 1555']
#not in both components in list
l_cat3_both=['BD-21  1074A', 'CD-35  6662', 'HD  37394', 'L   86-67', 'L  657-32',
 'LP  704-14', 'HD 147379B', '*  11 LMi', '*  12 Oph', '*  14 Her', '*  15 Sge',
 '*  18 Sco', '*  54 Psc', '*  61 UMa', '*  61 Vir', '*  70 Vir', '* 107 Psc',
 '* alf Aql', '* alf CMa', '* alf Cep', '* alf Lyr', '* bet Com', '* bet Leo',
 '* bet TrA', '* bet Vir', '* c Boo', '* e Vir', '* eps Ind', '* eta Crv',
 '* g Lup', '* gam Ser', '* iot Per', '* iot Psc', '* kap01 Cet', '* ksi Oph',
 '* mu. Vir', '* pi.03 Ori', '* psi Cap', '* psi Ser', '* psi05 Aur',
 '* sig Boo', '* sig Dra', '* tau Cet', '* w Her', 'RX J1942.2-2045',
 'BD+02   348', 'BD+05  1668', 'BD+05  3409', 'BD+18  2776', 'BD+28  4660',
 'BD+43  2796', 'BD+44  4548', 'BD+45  2247', 'BD+46  1635', 'BD+52   911',
 'BD+57  1274', 'BD+68   946', 'BD-12  4523', 'BD-18  3019', 'BD-18  3106',
 'BD-19  5899', 'CD-24  6144', 'CD-43 11010', 'CD-44 11909', 'CD-51 13128',
 'CPD-58  7400', 'CPD-64   303', 'G  21-13', 'G  22-26', 'G  44-43', 'G  46-2',
 'G  48-43', 'G  50-12', 'G  50-6', 'G  75-35', 'G  86-28', 'G  99-49',
 'G 110-2', 'G 112-50', 'G 123-55', 'G 125-30', 'G 129-47', 'G 131-62',
 'G 134-1', 'G 139-21', 'G 142-11', 'G 149-70', 'G 154-52', 'G 155-42',
 'G 166-49', 'G 170-62', 'G 173-39', 'G 186-24', 'G 188-49', 'G 190-17',
 'G 202-48', 'G 204-39', 'G 207-22', 'G 208-45', 'G 222-2', 'G 226-66',
 'G 233-42', 'G 239-25', 'StKM 1-852', 'GR*  50', 'EUVE J0625-60.0',
 'HD    166', 'HD   9540', 'HD  10436', 'HD  10780', 'HD  13445', 'HD  13579',
 'HD  17925', 'HD  21175', 'HD  21749', 'HD  23189', 'HD  25329', 'HD  29697',
 'HD  36003', 'HD  36395', 'HD  42581', 'HD  50281', 'HD  53143', 'HD  53680',
 'HD  64468', 'HD  65277', 'HD  65486', 'HD  67199', 'HD  69830', 'HD  73667',
 'HD  88230', 'HD 110315', 'HD 110810', 'HD 110833', 'HD 114837', 'HD 120780',
 'HD 125276', 'HD 127339', 'HD 128311', 'HD 131977', 'HD 135599', 'HD 142709',
 'HD 152705', 'HD 156384C', 'HD 157347', 'HD 157881', 'HD 166620', 'HD 170657',
 'HD 173818', 'HD 176029', 'HD 190067', 'HD 190771', 'HD 191391', 'HD 192263',
 'HD 199305', 'HD 200560', 'HD 200779', 'HD 202575', 'HD 207129', 'HD 211415',
 'HD 219134', 'HD 221503', 'HD 232979', 'HD 260655', 'L  242-66', 'L  320-124',
 'L  362-29', 'L  674-15', 'LP  755-19', 'L  901-10', 'LP  255-11',
 'LP  356-106', 'LP  374-39', 'LP  380-6', 'LP  390-16', 'LP  455-12',
 'LP  731-76', 'LP  775-31', 'LP  776-46', 'LP  870-65', 'LP  903-20',
 'LP  923-18', 'LP  993-115', 'LSPM J0140+2701', 'PM J01033-1347',
 'RX J2354.8+3831', 'CD-23 14742', 'Ross  614', 'Ross  989', 'SCR J1845-6357',
 'Smethells 22', 'RX J1935.4+3746', 'PM J23073-0415', 'UCAC3 129-126893',
 'LSPM J0738+2400', 'UCAC3 281-150921', 'V* AP Col', 'HD 197481', 'V* EV Lac',
 'V* GT Peg', 'V* V1386 Ori', 'V* V2215 Oph', 'V* V371 Ori', 'V* V816 Her',
 'V* YZ CMi', 'VB 16', 'WT 766']

#hwo
l_hwo= ['* tau Cet' ,'HD  17925' ,'* iot Per' ,'* zet01 Ret' ,'* kap01 Cet',
 '*  50 Per' ,'HD  50281' ,'*  22 Lyn' ,'HD  65907' ,'*  40 Leo' ,'*  61 UMa',
 '* eta Crv' ,'* bet Com' ,'*  61 Vir' ,'* gam Ser' ,'*  18 Sco' ,'HD 147513',
 '*  12 Oph' ,'HD 166620' ,'* psi Cap' ,'HD 207129' ,'* eps Ind' ,'V* TW PsA',
 'HD 219623' ,'HD 219482' ,'* gam Lep' ,'HD  46588' ,'*  11 LMi' ,'HD  90089',
 '*  20 Crt' ,'HD 114837' ,'V* V2215 Oph']
l_single=['* zet01 Ret', 'HD  50281', '* eps Ind']
l_hwo_sy=['* iot Hor', '* zet02 Ret' ,'* eps Eri' ,'* m Tau', '* bet CVn', '*  36 Oph',
 '* p Eri', '* p Eri' ,'* sig02 UMa', '*  36 Oph']
l_hwo_children=[]
l_wth=['* zet01 Ret']



In [ ]:
def detail_criteria(database_tables,table_names,l):
    """
    This code scans for reasons, why the given objects would not be included in cat 4
    
    """
    print('tbd: file starcat4 analysis where this code is used to give specific reason why a given object was not included into the cat4')
    cat_i=database_tables[table_names.index('ident')]
    cat_o=database_tables[table_names.index('objects')]
    cat_h=database_tables[table_names.index('best_h_link')]#-> use best_h_link
    children=[]
    not_found=[]
    number_of_stellar_siblings=[]
    multiple_parents=[]#multiple components in wds
    single_child=[]
    binary=[]
    higher_order_multiple=[]
    star_without_parent=[]
    system_without_child=[]
    for name in l:
        if len(cat_i['main_id'][np.where(cat_i['id']==name)])>0:
            main_id=cat_i['main_id'][np.where(cat_i['id']==name)][0]
            if cat_o['type'][np.where(cat_o['main_id']==main_id)][0]=='sy':
                if len(cat_h[np.where(cat_h['parent_main_id']==main_id)])>0:
                    #print('system object but with found child object',main_id)
                    #print(cat_h[np.where(cat_h['parent_main_id']==main_id)])
                    for child in cat_h['child_main_id'][np.where(cat_h['parent_main_id']==main_id)]:
                        children.append(child)
                else:
                    system_without_child.append(name)
            elif cat_o['type'][np.where(cat_o['main_id']==main_id)][0]=='st':
                #if it has parents:
                if len(cat_h[np.where(cat_h['child_main_id']==main_id)])>0:
                    #print('system object but with found child object',main_id)
                    #print(cat_h[np.where(cat_h['parent_main_id']==main_id)])
                    if len(cat_h[np.where(cat_h['child_main_id']==main_id)])>1:
                        multiple_parents.append(name)
                    elif len(cat_h[np.where(cat_h['child_main_id']==main_id)])==1:
                        st_sib=0
                        nestled=False
                        parent = cat_h['parent_main_id'][np.where(cat_h['child_main_id']==main_id)]
                            #wrong code here, parent can't be type st
                        for sibling in cat_h['child_main_id'][np.where(cat_h['parent_main_id']==parent)]:
                            print(cat_h[np.where(cat_h['child_main_id']==sibling)])
                            if cat_o['type'][np.where(cat_o['main_id']==sibling)][0]=='sy':
                                nestled=True
                            elif cat_o['type'][np.where(cat_o['main_id']==sibling)][0]=='st':
                                st_sib+=1
                            number_of_stellar_siblings.append(st_sib)
                        if st_sib==1 and nestled==False:
                            single_child.append(name)
                        if st_sib>2 or nestled==True:
                            higher_order_multiple.append(name)
                        if st_sib==2 and nestled==False:
                                binary.append(name)
                else:
                    star_without_parent.append(name)
            else:
                print('no sys or st \n',name,cat_o['main_id','type'][np.where(cat_o['main_id']==main_id)][0])
        else:
            #print('object not found',name)
            not_found.append(name)
    print('Of the',len(l), 'objects given:')
    print('Some are not in cat 4 because they are either:')
    print('system_without_child',system_without_child,len(system_without_child))
    print('star_without_parent',star_without_parent,len(star_without_parent))
    print('not found', not_found,'\n')
    print('Some were not included because they are systems and instead one should look at their child objects:')
    print('children',children,'\n')
    print('some are non trivial binaries:')
    print('multiple parents',multiple_parents,len(multiple_parents))
    print('higher_order_multiple',higher_order_multiple,len(higher_order_multiple),'\n')
    #print('# stellar siblings',number_of_stellar_siblings)
    if len(single_child)>0:
        print('single_child',single_child,len(single_child))
    print('And the reminder have conpanions that don t fit the spectral type requirements')
    print('trivial binary',binary,len(binary))
    return
detail_criteria(database_tables,table_names,l_cat3)

#'* 212 Pup A' kicked out in non trivial binaries in starcat but in this code as trivial binary with companion not matching sptype
#['* zet01 Ret', '*  50 Per', 'HD  50281', 'HD  65907', '* eps Ind', 'V* V2215 Oph'] other way round

### wrong classification for kick out

had all the hwo objects that were kicked out by cat4 criteria run through code to figure out exact reason and if was correct

Found issue with 
- 'HD  50281' which turned out to be correctly kicked out because of multiple parents
- * 212 Pup A was wrongly kicked out because of q.rd bug mes_h_link and is now correctly included
- '* zet01 Ret' correctly kicked out because companion has a child 

In [ ]:
import life_td_analysis as lta
importlib.reload(lta)#reload module after changing it

lta.show(provider=database_tables,table='star_basic',columns=['main_id','sptype_string','mass_st_value'],wherecol='main_id',\
    whereobj='FBS 1415+456')

In [ ]:
trivial_binaries=['* zet01 Ret']

In [ ]:
for name in trivial_binaries:
    main_id=cat_i['main_id'][np.where(cat_i['id']==name)][0]
    parent = cat_h['parent_main_id'][np.where(cat_h['child_main_id']==main_id)]
    print(cat_sb['main_id','sep_ang_value'][np.where(cat_sb['main_id']==parent)])
    for sibling in cat_h['child_main_id'][np.where(cat_h['parent_main_id']==parent)]:
        print(cat_sb['main_id','sptype_string'][np.where(cat_sb['main_id']==sibling)])

In [ ]:
#are single children single stars? or are spectroscopic binaries?
#* tau Cet 	WDS J01441-1556A
# topcat, wds, -> refs -> literature -> wide (very) low mass companions
# HD  17925 again a very low mass companion (17.5 mag)
#should include them in db but without sptype. why don't I have them?
name='* tau Cet'
main_id=cat_i['main_id'][np.where(cat_i['id']==name)][0]
parent=cat_h['parent_main_id'][np.where(cat_h['child_main_id']==main_id)]
print(cat_h[np.where(cat_h['parent_main_id']==parent)])

In [ ]:
import numpy as np #arrays
import pyvo as vo #catalog query
import astropy as ap #votables
from datetime import datetime
import importlib #reloading external functions after modification

#self created modules
import helperfunctions as hf
importlib.reload(hf)#reload module after changing it
import provider as p
importlib.reload(p)#reload module after changing it

table_names=['sources','objects','provider','ident','h_link','star_basic',
              'planet_basic','disk_basic','mes_mass_pl',
              'mes_teff_st','mes_radius_st','mes_mass_st','mes_binary','mes_sep_ang','best_h_link']
empty_provider=[ap.table.Table() for i in range(len(table_names))]

In [ ]:
wds=p.provider_wds(table_names,empty_provider[:],temp=True,test_objects=['WDS J01441-1556B'])
#why do I have binarity flag only in systems? want to have it in the stars too, change itin gaia provider too
#don't have h_link info included in gaia. 
#what do I actually need binarity flag for? sy already sais is multiple. st is clear that is multiple if it has a parent
#so maybe add binary flag to all objects that are st and have a parent. 
#binary flag is abbreviation of st object with parent

#next step: do partial creation, create db on dachs, and create new cat4. 
#expected change: more star_basic rows, stars get thrown out in missing paras and companions in not two in list
#not happened, filling level of binary_flag went up but objects still as single child

In [ ]:
#using partial create function to test if building best_h_link worked.
import importlib #reloading external functions after modification

#self created modules
import life_td as ltd
importlib.reload(ltd)#reload module after changing it

sim, gk, wds, exo, life, gaia, database_tables=ltd.partial_create(30,create=['gaia'])
#gaia error, might be because server down -> try again later -> working again

### double rows from id

In [ ]:
import importlib #reloading external functions after modification

#self created modules
import life_td_analysis as lta
importlib.reload(lta)#reload module after changing it

lta.show(database_tables,
         table='ident',
         columns=[],
         wherecol='id',
         whereobj='WDS J00008+1659A')
#1. won't want id_ref in there as is covered through id_source_idref

# database creation

In [2]:
import importlib #reloading external functions after modification

#self created modules
import life_td as ltd
importlib.reload(ltd)#reload module after changing it

#provider_tables_dict, database_tables= ltd.partial_create(30,create=['life'])

provider_tables_dict, database_tables= ltd.create_life_td(30)

#issue: does not wokr with loading as then id_source_idref already exists -> put in a workaround with a warning. though now did not happen...

#issue: create not working right now because exomercat server not available -> put in a workaround
#issue: main_id missing in star_basic -> suddenly working??

Building life_td data with distance cut of 30 pc
Trying to create SIMBAD tables from 2025-02-14...


 sorting object types...
removing 22  objects that had object types: ['BD?|MIR', 'BD?|MIR|NIR', 'BD?|MIR|NIR|Opt', 'BD?|NIR']
example object of them: WISEA J153429.75-104303.3


Trying to create Grant Kennedy Disks tables from 2024-02-09...
Trying to create WDS tables from 2025-02-14...
Creating  WDS  tables ...
 querying VizieR for WDS...
 performing distance cut...


Trying to create Exo-MerCat tables from 2025-02-14...


Trying to create LIFE tables from 2025-02-14...
            main_id             sptype_string class_lum class_temp class_temp_nr
------------------------------- ------------- --------- ---------- -------------
        2MASS J05171315-7706040             M         V          M             0
                      BPM 44589             M         V          M             0
                    CD-34  1169             M         V          M             0
                    CD-81   173             M         V          M             0
Cl* Melotte   22     MT     111            M:         V          M             0
                       G 111-52             M         V          M             0
                       G 242-59             K         V          K             0
                      G 273-171            MV         V          M             0
                      HD 170209         K(2)V         V          K             0
                      L   60-13             M         V      

Building sources table ...
Building objects table ...


Building provider table ...
Building ident table ...


Building best_h_link table ...
Building star_basic table ...
Building planet_basic table ...
Building disk_basic table ...
Building mes_mass_pl table ...
Building mes_teff_st table ...
Building mes_radius_st table ...
Building mes_mass_st table ...
Building mes_binary table ...
Building mes_sep_ang table ...
Building h_link table ...


Unifying null values...
Saving data...


In [ ]:
from time import time,ctime

print(ctime(time()))

#most likely can gain performance by differentiating between single object and multiple object groups

#esier method for performance assertment is to print time at start and end of function

#to improve performance

import profile
profile.run('main()')

# load and analyze database

In [48]:
import importlib #reloading external functions after modification

#self created modules
import life_td as ltd
importlib.reload(ltd)#reload module after changing it

provider_tables_dict, database_tables= ltd.load_life_td()


Loading life_td generated data


Finding issue that causes M0-M5 stars to have too high temperatures (4250 instead of 3000)

In [42]:
print(database_tables)#from create is a dict, from load is a list

{'sources': <Table length=102>
                       ref                            provider_name    source_id
                      object                              object         int64  
-------------------------------------------------- ------------------- ---------
                               1967AJ.....72.1334C              SIMBAD         1
                               1980AJ.....85..454H              SIMBAD         2
                               1982A&A...108..406K              SIMBAD         3
                               1982ApJ...253..237C              SIMBAD         4
                               1982MSS...C03....0H              SIMBAD         5
                               1983SAAOC...7..106W              SIMBAD         6
                               1984ApJ...276..602G              SIMBAD         7
                               1985ApJS...59..197B              SIMBAD         8
                               1988AJ.....95.1841G              SIMBAD        

In [53]:
import numpy as np
#9=mesteff
#5=starbasic
#print(database_tables[5].colnames)
cols=['main_id','sptype_string','class_temp_nr','class_temp','teff_st_value','teff_st_source_idref']
mstars=database_tables['star_basic'][cols][np.where(database_tables['star_basic']['class_temp']=='M')]
mstars_tempgiven=mstars[np.where(mstars['teff_st_value']!=1e+20)]
print(mstars_tempgiven[np.where(mstars_tempgiven['teff_st_value']>4000)])

 main_id   sptype_string class_temp_nr ... teff_st_value teff_st_source_idref
                                       ...       K                           
---------- ------------- ------------- ... ------------- --------------------
 L  372-58         M5.5V           5.5 ...        4250.0                   93
LP  731-58         M6.5V           6.5 ...        4250.0                   93
 Ross  248         M5.0V           5.0 ...        4250.0                   93
 V* YZ Cet        M4.0Ve           4.0 ...        4250.0                   93


In [52]:
print(database_tables['sources'][np.where(database_tables['sources']['source_id']==93)])

            ref              provider_name source_id
---------------------------- ------------- ---------
2022arXiv220800211G GSP-Spec          Gaia        93


In [54]:
#so lets find out if in gaia archive this is also the case
#first find gaia id
print(database_tables['ident'][np.where(database_tables['ident']['main_id']=='L  372-58')])
#Gaia DR3 4848140361962951552

#on archive only gspphot showable and it is empty

             id               main_id  id_source_idref object_idref
---------------------------- --------- --------------- ------------
     2MASS J03355969-4430453 L  372-58              19          149
            APMPM J0336-4431 L  372-58              19          149
                    CNS5 919 L  372-58              19          149
                  GCRV 26222 L  372-58              19          149
            GEN# +9.85995046 L  372-58              19          149
                     GJ 1061 L  372-58              19          149
Gaia DR2 4848140361962951552 L  372-58              19          149
Gaia DR3 4848140361962951552 L  372-58              19          149
                   L  372-58 L  372-58              18          149
                   L  372-58 L  372-58              19          149
                    LFT  295 L  372-58              19          149
                   LHS  1565 L  372-58              19          149
                  LP  995-46 L  372-58          

In [60]:
from sdata import empty_dict
from provider.utils import fetch_main_id, IdentifierCreator, fill_sources_table, create_sources_table, ids_from_ident, replace_value, create_provider_table, query
from pyvo.dal import TAPService

adql_query="""
    SELECT top 10 s.source_id ,p.mass_flame, p.radius_flame,
        p.teff_gspphot, p.teff_gspspec, m.nss_solution_type, p.age_flame
    FROM gaiadr3.gaia_source as s
        JOIN gaiadr3.astrophysical_parameters as p ON s.source_id=p.source_id
            LEFT JOIN gaiadr3.nss_two_body_orbit as m ON s.source_id=m.source_id
            WHERE s.source_id = 4848140361962951552"""
gaia = empty_dict.copy()
gaia['provider'] = create_provider_table('Gaia',
                                  "https://gea.esac.esa.int/tap-server/tap",
                                  '2016A&A...595A...1G')
try: 
    gaia_helptab=query(gaia['provider']['provider_url'][0],adql_query) 

except:
    #because of bug in gaia server where async not working currently
    service = TAPService(gaia['provider']['provider_url'][0])
    result=service.run_sync(adql_query.format(**locals()), maxrec=160000)
    gaia_helptab=result.to_table()


Trying to create Gaia tables from 2025-02-10...


In [62]:
print(gaia_helptab['source_id','teff_gspspec'])

     source_id      teff_gspspec
                         K      
------------------- ------------
4848140361962951552       4250.0


In [ ]:
#so yes that is how it is in gaia. now find out if this parameter is reliable and if not lower its quality

In [ ]:
import analysis as lta

lta.sanity_tests(database_tables,5)

# database example query

In [ ]:
import importlib #reloading external functions after modification
import sys

sys.path.append('examples') 
#self created modules
import life_td_access_example as test_cat
importlib.reload(test_cat)#reload module after changing it

#print(test_cat.example_query())

adql_query="""
    SELECT o.main_id, o.type, sb.dist_st_value,  
    sb.class_temp, o_parent.main_id AS parent_main_id, sb_parent.sep_ang_value, sb.class_lum
    FROM life_td.star_basic AS sb
    JOIN life_td.object AS o ON sb.object_idref=o.object_id
    LEFT JOIN life_td.h_link AS h ON o.object_id=h.child_object_idref
    LEFT JOIN life_td.object AS o_parent ON h.parent_object_idref=o_parent.object_id
    LEFT JOIN life_td.star_basic AS sb_parent ON o_parent.object_id=sb_parent.object_idref
    WHERE o.main_id = 'HD  15468'"""
    #    WHERE o.type = 'st' AND sb.class_lum IS NOT NULL AND sb.dist_st_value < """+str(20) 
    
adql_query2="""SELECT main_id as Child_main_id, object_id as child_object_id
FROM life_td.h_link
JOIN life_td.ident as p on p.object_idref=parent_object_idref
JOIN life_td.object on object_id=child_object_idref
WHERE p.id = 'HD  15468'"""

print(test_cat.example_query(adql_query2))

In [ ]:
import importlib #reloading external functions after modification
import numpy as np

#self created modules
import analysis as lta
importlib.reload(lta)#reload module after changing it

testobject=188
#(755/6/7) gaia -> 1000 mass values, 3800 radius, 3800/5000 teff
#(433) life -> 6400 mass values, 6400 teff, 6400 radius
#total mass_st 7500, raidus 10 000, 15300
temp =database_tables[table_names.index('ident')]
temp2=temp['main_id','id_source_idref'][np.where(temp['id_source_idref']!=172)]
print(temp2)
temp3=temp2[np.where(temp2['id_source_idref']!=171)]
print(temp3)


lta.show(database_tables,
         table='sources',
         columns=[],
         wherecol='source_id',
         whereobj=testobject)




In [ ]:
lta.show(database_tables,
         table='best_h_link',
         columns=['parent_object_idref','child_object_idref'],
         wherecol='child_object_idref',
         whereobj=360)
lta.show(database_tables,
         table='best_h_link',
         columns=['parent_object_idref','child_object_idref'],
         wherecol='parent_object_idref',
         whereobj=14673)

In [ ]:
#see about their separation measurements. if there are none in the last few years or they are very distant 
#than I can see them as non higher order
lta.show(database_tables,
         table='mes_sep_ang',
         columns=['sep_ang_value','sep_ang_obs_date'],
         wherecol='object_idref',
         whereobj=14676)



# Non physical higher order multiples in the database
Jens raised the issue that I have eta cas classified as higher order multiple despite it most likely being a trivial binary system with background objects. I therefore checked my data and found out that in WDS it has components from A to J. I wrote an email to WDS asking how I can distinguish physical from not bound companions. A look at the most resent separations listened revealed that I couldn't just distinguish them by how outdated their observations are or how distant the companion is.

- AB: 13.4 in 2022
- AC: 225 in 2000
- AD: 189.4 in 2011
- AE: 75.6 2015
- 378.3 2000
- 419.7 2012
- 701.1 2012
- 90.3 2012
- 237.1 2012


In [ ]:
#* eta Cas 360, WDS J00491+5749A
#child of eta cas: sdb-v2-004906.29+574854.7 20110
#parents of eta cat:
#14673 WDS J00491+5749AB
#14674 WDS J00491+5749AC
#14675 WDS J00491+5749AD
#14676 WDS J00491+5749AE
#.. 81 WDS J00491+5749AJ

# * eta Cas B 361 WDS J00491+5749B

# getting comfortable with classes

to do next: go over docstring of structured_data_class.py and ingest more of the tables and parameters needed

In [ ]:
import importlib #reloading external functions after modification

import structured_data_class as sdc
importlib.reload(sdc)#reload module after changing it
import helperfunctions as hf
importlib.reload(hf)

In [ ]:
#now getting stuff replaced. careful, I changed order of tables by exchanging h_link with best_h_link
#try saving with empty_ as object name
empty=sdc.provider('empty')
#now replace in building function init with one of provider class but when accessing any init table make sure goes
#through provider class and not [0] because of different order of tables
#now next task is to run database creation and compare output with the one from before

In [ ]:
pre_sdc=hf.load(['sources','objects','provider','ident','h_link','star_basic',
              'planet_basic','disk_basic','mes_mass_pl',
              'mes_teff_st','mes_radius_st','mes_mass_st','mes_binary',
             'mes_sep_ang','best_h_link'],stringtoobjects=True,location=additional_data_path+'pre_sdc/')

In [ ]:
empty=sdc.provider('empty')
empty.table_names

In [ ]:
post_sdc=hf.load(['sources','objects','provider','ident','h_link','star_basic',
              'planet_basic','disk_basic','mes_mass_pl',
              'mes_teff_st','mes_radius_st','mes_mass_st','mes_binary',
             'mes_sep_ang','best_h_link'],stringtoobjects=True,location='../../data/')
ids='main_id'
paras=['plx_value']
i=empty.table_names.index('star_basic')
hf.compare_catalogs(pre_sdc[i],post_sdc[i],ids,ids,paras,paras)
#there might be differences in the ids parameter

#stuff the same in object type despite me having forgotten to add it to the class

In [ ]:
from datetime import datetime
import astropy as ap

In [ ]:
test=sdc.provider('test')
print(test.table('provider'))

access=datetime.now().strftime('%Y-%m-%d')
test.table('provider').add_row(['SIMBAD',"http://simbad.u-strasbg.fr:80/simbad/sim-tap",
                                '2000A&AS..143....9W',access])

#find out how to add only element wise not whole row


print(test.table('provider'))
#find out error message for one of the columns empty in join
col=ap.table.MaskedColumn(name='empty',dtype=object, length=len(test.table('provider')))
test.table('provider').add_column(col)
print(test.table('provider'))

In [ ]:
simbad=sdc.provider('simbad')
print(simbad.table('provider'))
simbad.table('provider').add_row()
print(simbad.table('provider'))

In [ ]:
simbad.table('provider')['provider_name']=['SIMBAD']
print(simbad.table('provider'))

In [ ]:
simbad.table('provider')['provider_url']=["http://simbad.u-strasbg.fr:80/simbad/sim-tap"]
print(simbad.table('provider'))

In [ ]:
test.list_of_tables

In [ ]:
simbad.list_of_tables

In [ ]:
#now test working with two different ones and see if it is ok
simbad.list_of_tables

In [ ]:
simbad=initialized_structure.copy('simbad')

In [ ]:
sim.provider

In [ ]:
sim.list_of_tables

# solved stuff

### no lum class given no V assumed -> solved -> implemented as test

### more than V assigned model values -> solved

In [ ]:
import importlib
import analysis as lta
importlib.reload(lta)

lta.show(database_tables,
         table='star_basic',
         columns=['main_id','sptype_string','class_lum','mass_st_value','mass_st_source_idref'],
         wherecol='class_lum',
         whereobj='IV')
#oh thos mass values could come from other sources

### not 100% fill level binary_flag -> solved

In [ ]:
cat_sb=database_tables[table_names.index('star_basic')]
#cat_sb=gaia[table_names.index('mes_binary')]
singles=cat_sb[np.where(cat_sb['binary_flag']!='True')]
problematic=singles[np.where(singles['binary_flag']!='False')]
print(problematic['main_id','binary_flag'])
print(cat_sb)
#issue is, that gaia not sy don't have binary_flag entry currently
#now there could be another issue now. what about gaia stars that are singles by gaia but multiples by wds? -> lower gaia quality

### starcat4 has objects with parents that have binary flag = False -> solved

In [ ]:
cat_sb=database_tables[table_names.index('star_basic')]
cat_mb=database_tables[table_names.index('mes_binary')]
cat_s=database_tables[table_names.index('sources')]

#* 110 Her
print(cat_sb['main_id','binary_flag','binary_source_idref'][np.where(cat_sb['main_id']=='* 110 Her')])
print(cat_mb['main_id','binary_flag','binary_source_idref'][np.where(cat_mb['main_id']=='* 110 Her')])
print(cat_s[np.where(cat_s['source_id']==171)])

### dM2.5 gets not matched with mamajek M2.5V -> solved

In [ ]:
import analysis as lta
importlib.reload(lta)#reload module after changing it

lta.show(provider=database_tables,table='star_basic',columns=['main_id','sptype_string','mass_st_value','class_lum'],wherecol='main_id',\
    whereobj='FBS 1415+456')
#G 155-42

#one issue was that in match sptype only looked at first two entries of sptype_string (dM) and tried to match that to mamajek

## disk update -> solved, just copy paste provider code (and remove source param)

In [ ]:
#disks from grant
import numpy as np #arrays
import pyvo as vo #catalog query
import astropy as ap #votables


gk_disks=ap.io.votable.parse_single_table(
        "../../data/additional_data/Grant_absil_2013_.xml").to_table()

print(gk_disks.colnames)
gk_disks_new=ap.io.votable.parse_single_table(
        "../../data/additional_data/sdb_30pc_09_02_2024.xml").to_table()
print(gk_disks_new.colnames)
#error module 'astropy' has no attribute 'io' works if I import vo and np too
#error -> found out -> asked grant about it.

print(gk_disks['main_id'])
print(gk_disks_new['main_id'])

print('parameter, old type, new type')
for col in gk_disks.colnames:
    if type(gk_disks[col][0])!=type(gk_disks_new[col][0]):
        print(col, type(gk_disks[col][0]),type(gk_disks_new[col][0]))
#okay, whole lot of types changed.

para='e_rdisk_bb'#'plx_value'#'rdisk_bb'
print(gk_disks[para][np.where(gk_disks[para]=='None')])
#print(gk_disks[para].mask.nonzero()[0]) 'Column' object has no attribute 'mask'
print(gk_disks_new[para].mask.nonzero()[0])
for i in range(15):
    print(gk_disks_new[para][i*10:i*10+10])#[np.where(gk_disks_new['rdisk_bb']==float('nan'))])
#no nan, no masked,...

In [ ]:
import utils as hf
from provider import fetch_main_id, sources_table,replace_value

provider_gk(table_names,empty_provider[:],30)
#errors I get:
#wit old file
#gk_disks['plx_value']=gk_disks['plx_value'].filled('None') 'Column' object has no attribute 'filled'
#-> deleted that row and then it worked
#with new file
#gk_disks['plx_value']=gk_disks['plx_value'].filled('None') Cannot set fill value of string with array of dtype float64

cleaning up life_td.py

In [ ]:
    if 'sim' in create:
        cat=provider_simbad(distance_cut_in_pc)
    else: 
        cat = load_cat('sim')
    provider_tables_dict['sim']=cat

    if 'sdb' in create:
        cat=provider_sdb(distance_cut_in_pc)
    else: 
        cat = load_cat('sdb')
    provider_tables_dict['sdb']=cat

    if 'wds' in create:
        cat=provider_wds(False)
    else: 
        cat = load_cat('wds')
    provider_tables_dict['wds']=cat
  
    if 'exo' in create:
        cat=provider_exo(temp=True)
    else: 
        cat = load_cat('exo')
    provider_tables_dict['exo']=cat
  
    if 'life' in create:
        cat=provider_life()
    else: 
        cat = load_cat('life')
    provider_tables_dict['life']=cat
    
   
    if 'gaia' in create:
        cat=provider_gaia(distance_cut_in_pc)
    else: 
        cat = load_cat('gaia')
    provider_tables_dict['gaia']=cat